In [3]:
# %load /home/hwixley/Documents/Year3/IAML/cwk2/INFR10069-2020-CW2/helpers/iaml01cw2_helpers.py
#
#  Helper functions for IAML 2020/21 cw2
#
import os
import gzip
import numpy as np

#
#  This function is after  https://github.com/zalandoresearch/fashion-mnist
#
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

#
#------ IAML original functions ---------------
#
def load_FashionMNIST(data_path):
    Xtrn, Ytrn = load_mnist(data_path, "train")
    Xtst, Ytst = load_mnist(data_path, "t10k")
    return Xtrn.astype(np.float), Ytrn, Xtst.astype(np.float), Ytst

def load_CoVoST2(data_path):
    data = np.load(data_path+'/speech_trn.npz', allow_pickle=True)
    Xtrn, Ytrn = data['Xtrn'], data['Ytrn']
    data = np.load(data_path+'/speech_tst.npz', allow_pickle=True)
    Xtst, Ytst = data['Xtst'], data['Ytst']
    return Xtrn, Ytrn, Xtst, Ytst


In [46]:

##########################################################
#  Python script template for Question 1 (IAML Level 10)
#  Note that
#  - You should not change the filename of this file, 'iaml01cw2_q1.py', which is the file name you should use when you submit your code for this question.
#  - You should define the functions shown below in your code.
#  - You can define function arguments (parameters) and returns (attributes) if necessary.
#  - In case you define helper functions, do not define them here, but put them in a separate Python module file, "iaml01cw2_my_helpers.py", and import it in this script.
#  - For those questions requiring you to show results in tables, your code does not need to present them in tables - just showing them with print() is fine.
#  - You do not need to include this header in your submission
##########################################################

#--- Code for loading the data set and pre-processing --->
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data:
Xtrn, Ytrn, Xtst, Ytst = load_FashionMNIST("/home/hwixley/Documents/Year3/IAML/cwk2/INFR10069-2020-CW2/data")
Xtrn_orig = Xtrn.copy
Xtst_orig = Xtst.copy
Xtrn = Xtrn/255
Xtst = Xtst/255

Xmean = Xtrn.mean(0)
Xtrn_nm = Xtrn - Xmean
Xtst_nm = Xtst - Xmean
#<----

# Q1.1
def iaml01cw2_q1_1():
    print("First 4 elements of the first training sample in Xtrn_nm:")
    print(Xtrn_nm[0,0:4])
    print("\nFirst 4 elements of the last training sample in Xtrn_nm:")
    print(Xtrn_nm[-1,0:4])
#
# iaml01cw2_q1_1()   # comment this out when you run the function

# Q1.2
def iaml01cw2_q1_2():
    # Array of classes
    classes = np.unique(Ytrn)
    
    # Initialisation of variables
    classSamples = []
    means = []
    closest = []
    furthest = []
    classMeans = np.zeros((10,784))
    
    # Initialise empty nested arrays
    for c in classes:
        classSamples.append([])
        closest.append([])
        furthest.append([])
    
    # Add all the indexes of samples for a given classs in the classSamples list
    for row in range(Ytrn.size): 
        classSamples[Ytrn[row]].append(row)
    
    # Iterates over all classes (to calculate means, closest and furthest samples)
    for c in range(10):
        #Represents the relevant samples for the given class c
        samples = classSamples[c]
        
        #Calculate the mean samples for each class
        for index in samples:
            classMeans[c] += Xtrn[index,:]
            
        classMeans[c] = classMeans[c]/len(samples)
        meanC = classMeans[c].reshape(28,28)
        means.append(meanC)
        
        #Calculate closest samples variables
        close1 = -1
        closeDist1 = 10000
        close2 = -1
        closeDist2 = 10000
        #Calculate furthest samples variables
        far1 = -1
        farDist1 = 0
        far2 = -1
        farDist2 = 0
        
        #Iterate through the samples of the given class to find the closest & furthest samples
        for index in samples:
            dist = sum((classMeans[c]-Xtrn[index,:])**2)
            
            if dist < closeDist1:
                close1 = index
                closeDist1 = dist
            elif dist < closeDist2:
                close2 = index
                closeDist2 = dist
                
            if dist > farDist1:
                far1 = index
                farDist1 = dist
            elif dist > farDist2:
                far2 = index
                farDist2 = dist
        
        #Store the closest and furthest samples
        closest[c].append(close1)
        closest[c].append(close2)
        furthest[c].append(far1)
        furthest[c].append(far2)
    
    #Create a subplot to show the relevant samples
    fig, axs = plt.subplots(10,5,figsize=(16,16))
    fig.tight_layout(h_pad=2.5)
    for i in range(10):
        #Mean
        axs[i,0].imshow(means[i], cmap="gray_r")
        axs[i,0].set_ylabel("Class " + str(i) + "\n",fontsize=20)
        axs[i,0].set_xticklabels([])
        axs[i,0].set_yticklabels([])
        
        for n in range(1,3):
            #Closest
            axs[i,n].imshow(Xtrn[closest[i][n-1],:].reshape(28,28), cmap="gray_r")
            axs[i,n].set_title("Sample " + str(closest[i][n-1]),fontsize=18)
            axs[i,n].set_xticklabels([])
            axs[i,n].set_yticklabels([])
            
            #Furthest
            axs[i,n+2].imshow(Xtrn[furthest[i][-n+2],:].reshape(28,28), cmap="gray_r")
            axs[i,n+2].set_title("Sample " + str(furthest[i][-n+2]),fontsize=18)
            axs[i,n+2].set_xticklabels([])
            axs[i,n+2].set_yticklabels([])
    
    #Add appropriate axis labels to the subplot
    axs[9,0].set_xlabel("\nMean sample",fontsize=20)
    axs[9,1].set_xlabel("\nClosest sample\nto mean",fontsize=20)
    axs[9,2].set_xlabel("\n2nd closest sample\nto mean",fontsize=20)
    axs[9,3].set_xlabel("\n2nd furthest sample\nfrom mean",fontsize=20)
    axs[9,4].set_xlabel("\nFurthest sample\nfrom mean",fontsize=20)
            
                
#
# iaml01cw2_q1_2()   # comment this out when you run the function

# Q1.3
def iaml01cw2_q1_3():
    from sklearn.decomposition import PCA
    
    pca = PCA()
    pca.fit(Xtrn_nm)
    print("The cumulative explained variance:")
    print(round(pca.explained_variance_.sum(),3))
    print("\nThe explained variances for the first 5 principal components:")
    
    for i in range(5):
        print("PC " + str(i) + " = " + str(round(pca.explained_variance_[i],3)))
    
    
#
# iaml01cw2_q1_3()   # comment this out when you run the function


# Q1.4
def iaml01cw2_q1_4():
    from sklearn.decomposition import PCA
    
    pca = PCA()
    pca.fit(Xtrn_nm)
    y = pca.explained_variance_ratio_.cumsum()
    x = np.arange(1,785)
    
    plt.plot(x,y)
    plt.grid(True)
    plt.xlabel("Attribute #")
    plt.ylabel("Cumulative explained variance ratio")
    plt.title("A graph to show the cumulative explained variance\nratio for the attributes in 'Xtrn_nm'")
#
# iaml01cw2_q1_4()   # comment this out when you run the function


# Q1.5
def iaml01cw2_q1_5():
    from sklearn.decomposition import PCA
    
    pca = PCA()
    pca.fit(Xtrn_nm)
    
    fig, axs = plt.subplots(2,5,figsize=(15,15))
    index = 0
    for r in range(2):
        for c in range(5):
            axs[r,c].imshow(pca.components_[index].reshape(28,28))
            axs[r,c].set_title("PC " + str(index),fontsize=18)
            index += 1

#
# iaml01cw2_q1_5()   # comment this out when you run the function


# Q1.6
def iaml01cw2_q1_6():
    from sklearn.decomposition import PCA
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    
    classes = np.unique(Ytrn)
    K = [5,20,50,200]
    orig_classSamples = np.empty((10,784))
    
    # Get indexes of the first samples for each class
    for c in classes:
        for s in range(Ytrn.size):
            if Ytrn[s] == c:
                orig_classSamples[c,:] = Xtrn_nm[s,:]
                break
    
    #Numpy matrix to store all the RMSE scores
    rmse = np.empty((10,4))
    
    #Iterate through the different values of K
    for i in range(4):
        k = K[i]
        new_classSamples = np.empty((10,784))
        
        #Iterate through the different classes
        for s in range(10):
            pca = PCA(n_components=k)
            pca.fit(Xtrn_nm)
            X = orig_classSamples[s,:]
            newX = pca.transform(X.reshape(1,-1))
            
            inverseX = pca.inverse_transform(newX).reshape(-1,1)[:,0]
            rmse[s,i] = sqrt(mean_squared_error(X,inverseX))
    
    print(np.round(rmse,3))
            
            
#
# iaml01cw2_q1_6()   # comment this out when you run the function


# Q1.7
def iaml01cw2_q1_7():
    from sklearn.decomposition import PCA
    
    classes = np.unique(Ytrn)
    K = [5,20,50,200]
    orig_classSamples = np.empty((10,784))
    
    # Get the first samples for each class
    for c in classes:
        for s in range(Ytrn.size):
            if Ytrn[s] == c:
                orig_classSamples[c,:] = Xtrn_nm[s,:]
                break
    
    #Variables to store the inversely transformed samples
    index1 = 0
    new_classSamples = np.empty((40,784))
    
    #Iterate through the values of k
    for i in range(4):
        k = K[i]
        print(i)
        
        #Iterate through the classes
        for s in range(10):
            pca = PCA(n_components=k)
            pca.fit(Xtrn_nm)
            X = orig_classSamples[s,:]
            newX = pca.transform(X.reshape(1,-1))
            
            inverseX = pca.inverse_transform(newX).reshape(-1,1)[:,0]
            new_classSamples[index1,:] = inverseX
            index1 += 1
    
    #Put all of these images into a subplot
    fig, axs = plt.subplots(10,4,figsize=(18,18))
    index = 0
    for k in range(4):
        for c in range(10):
            if k == 0:
                axs[c,k].set_ylabel("Class " + str(c),fontsize=20)
                
            axs[c,k].imshow(new_classSamples[index,:].reshape(28,28))
            axs[c,k].set_xticklabels([])
            axs[c,k].set_yticklabels([])
            index += 1
            
    axs[9,0].set_xlabel("K = 5",fontsize=20)
    axs[9,1].set_xlabel("K = 20",fontsize=20)
    axs[9,2].set_xlabel("K = 50",fontsize=20)
    axs[9,3].set_xlabel("K = 2000",fontsize=20)
    
    plt.show()
#
# iaml01cw2_q1_7()   # comment this out when you run the function


# Q1.8
def iaml01cw2_q1_8():
    from sklearn.decomposition import PCA
    import matplotlib
    import matplotlib.cm as cm
    
    pca = PCA(n_components=2)
    pca.fit(Xtrn_nm)
    newXtrn = pca.transform(Xtrn_nm)
    
    x = newXtrn[:,0]
    y = newXtrn[:,1]
    z = np.empty((60000,4)).reshape(-1,1)
    classes = np.unique(Ytrn)
    colours = np.empty((10))
    
    norm = matplotlib.colors.Normalize(vmin=min(classes),vmax=max(classes),clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap="coolwarm")
        
    #print(plt.get_cmap("coolwarm").count)
    for s in range(Ytrn.size):
        z[s,:] = mapper.to_rgba(Ytrn[s])
    
    plt.scatter(x,y,c=z,cmap="coolwarm")
#
# iaml01cw2_q1_8()   # comment this out when you run the function


ValueError: cannot copy sequence with size 4 to array axis with dimension 1